<a href="https://colab.research.google.com/github/Thabetiphanipriyagoud/NLP-PROJECT/blob/main/LSTM%26BERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install Tokenizer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 79 kB 3.2 MB/s 


In [ ]:
!pip install pad_sequences

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for pad-sequences: filename=pad_sequences-0.6.1-py3-none-any.whl size=10216 sha256=0ca97910ab50fef7934851bfce9948f6d2017c0704c680e4328b7d151d2055a7
  Stored in directory: /root/.cache/pip/wheels/3f/80/05/48e28be7b6bde8b3378f747f50bc32a87cb281c2e89ae74500
Successfully built pad-sequences


In [ ]:
import tokenizer

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns 
from scipy import stats


from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding,LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.layers import Dropout
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.models import load_model
from sklearn.model_selection import train_test_split

import re
import nltk 
nltk.download("stopwords")
from nltk.corpus import stopwords
nltk.download('punkt')

import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/trainPhani.csv")

In [ ]:
data=data.drop(['discourse_id','essay_id'],axis='columns')
data.head()

,discourse_text,discourse_type,discourse_effectiveness
0,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,Adequate
1,"On my perspective, I think that the face is a ...",Position,Adequate
2,I think that the face is a natural landform be...,Claim,Adequate
3,"If life was on Mars, we would know by now. The...",Evidence,Adequate
4,People thought that the face was formed by ali...,Counterclaim,Adequate


In [ ]:
data.shape

(36765, 3)

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36765 entries, 0 to 36764
Data columns (total 3 columns):
 #   Column                   Non-Null Count  Dtype 
---  ------                   --------------  ----- 
 0   discourse_text           36765 non-null  object
 1   discourse_type           36765 non-null  object
 2   discourse_effectiveness  36765 non-null  object
dtypes: object(3)
memory usage: 861.8+ KB


In [ ]:
data.describe()

,discourse_text,discourse_type,discourse_effectiveness
count,36765,36765,36765
unique,36691,7,3
top,Summer projects should be student-designed,Evidence,Adequate
freq,14,12105,20977


In [ ]:
new_label = {"discourse_effectiveness": {"Ineffective": 0, "Adequate": 1, "Effective": 2}}
data = data.replace(new_label)
data = data.rename(columns = {"discourse_effectiveness": "label"})
data.head()

,discourse_text,discourse_type,label
0,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,1
1,"On my perspective, I think that the face is a ...",Position,1
2,I think that the face is a natural landform be...,Claim,1
3,"If life was on Mars, we would know by now. The...",Evidence,1
4,People thought that the face was formed by ali...,Counterclaim,1


In [ ]:
data.head()

,discourse_text,discourse_type,label
0,"Hi, i'm Isaac, i'm going to be writing about h...",Lead,1
1,"On my perspective, I think that the face is a ...",Position,1
2,I think that the face is a natural landform be...,Claim,1
3,"If life was on Mars, we would know by now. The...",Evidence,1
4,People thought that the face was formed by ali...,Counterclaim,1


In [ ]:
def gen_freq(text):
    #will store all the words in list
    words_list = []
    
    #Loop over all the words and extract word from list
    for word in text.split():
        words_list.extend(word)
        
    #Generate word frequencies using value counts in word_list
    word_freq = pd.Series(words_list).value_counts()
    
    #print top 100 words
    word_freq[:100]
    
    return word_freq   

In [ ]:
freq = gen_freq(data.discourse_text.str)
freq

the            77217
to             57242
a              38013
and            36578
of             33395
               ...  
cluping            1
clup               1
leasat             1
coumputer's        1
someones'          1
Length: 53022, dtype: int64

In [ ]:
from nltk.corpus import stopwords
stop_word_list = stopwords.words('english')
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.tokenize.toktok import ToktokTokenizer

In [ ]:
tokenizer=ToktokTokenizer()

In [ ]:
def remove_stopwords(text, is_lower_case=False):
    tokens = tokenizer.tokenize(text)
    tokens = [token.strip() for token in tokens]
    if is_lower_case:
        filtered_tokens = [token for token in tokens if token not in stop_word_list]
    else:
        filtered_tokens = [token for token in tokens if token.lower() not in stop_word_list]
    filtered_text = ' '.join(filtered_tokens)    
    return filtered_text
#Apply function on review column
data['discourse_text']= data['discourse_text'].apply(remove_stopwords)

In [ ]:
import re
#clearing punctuation & unnecessary marks
data['discourse_text'] = data['discourse_text'].apply(lambda x: re.sub('[,\.!?:()"]', '', x))
data['discourse_text'] = data['discourse_text'].apply(lambda x: re.sub('[^a-zA-Z"]', ' ', x))

#capitalization to lowercase
data['discourse_text'] = data['discourse_text'].apply(lambda x: x.lower())

#cleaning extra spaces
data['discourse_text'] = data['discourse_text'].apply(lambda x: x.strip())

In [ ]:
from bs4 import BeautifulSoup

In [ ]:
#Removing the html strips
def strip_html(text):
    soup = BeautifulSoup(text, "html.parser")
    return soup.get_text()

#Removing the square brackets
def remove_between_square_brackets(text):
    return re.sub('\[[^]]*\]', '', text)

#Removing the noisy text
def denoise_text(text):
    return text

#Apply function on review column
    text = strip_html(text)
    text = remove_between_square_brackets(text)
data['discourse_text']=data['discourse_text'].apply(denoise_text)

In [ ]:
data_copy = data.copy()

In [ ]:
label = data['label'].values
label

array([1, 1, 1, ..., 1, 0, 0])

In [ ]:
data['label'].unique()

array([1, 0, 2])

In [ ]:
data = data['discourse_text']

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(data,label,test_size = 0.2, random_state = 42)

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words = 15000)
tokenizer.fit_on_texts(data)
#tokenizer.word_index

In [ ]:
x_train_tokens = tokenizer.texts_to_sequences(x_train)
x_test_tokens = tokenizer.texts_to_sequences(x_test)

In [ ]:
#Then we take the word count of each of our sentences in our data and create a list.
num_tokens = [len(tokens) for tokens in x_train_tokens + x_test_tokens]
num_tokens = np.array(num_tokens)

In [ ]:
#Here, when setting the number of tokens, a number is determined by taking into account the variability around the average.
max_tokens = np.mean(num_tokens) + 2 * np.std(num_tokens)
max_tokens = int(max_tokens)
max_tokens

68

In [ ]:
#It is checked what percentage of the data this determined number covers.
np.sum(num_tokens < max_tokens) / len(num_tokens)

0.9512307901536787

In [ ]:
#data is adjusted according to the number of tokens specified
x_train_pad = pad_sequences(x_train_tokens, maxlen=max_tokens)
x_test_pad = pad_sequences(x_test_tokens, maxlen=max_tokens)
x_train_pad.shape

(29412, 68)

In [ ]:
idx = tokenizer.word_index
inverse_map = dict(zip(idx.values(), idx.keys()))

def return_to_sentence(tokens):
    words = [inverse_map[token] for token in tokens if token!=0]
    text = ' '.join(words)
    return text

In [ ]:
#normal comment
print(return_to_sentence(x_train_pad[9]))

shows nasa trying keep sercet head mouth eyes noes ears mars trying put type effort get deeper situation even assummed normal planet nobody lived unusal head poped nowhere planet mars


In [ ]:
#token equivalent of comment
print(x_train_pad[9])

[    0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0   235   129   190    93 10420   768   842   512 10421  6179
    76   190   142   518   730    11  2860   292    21 10422   985    67
  1001  1337  6690   768 10423  3146    67    76]


In [ ]:
from tensorflow.python.keras.optimizer_v2.rmsprop import RMSProp
model = Sequential()

embedding_size = 50

model.add(Embedding(input_dim=15000,output_dim=embedding_size,input_length=max_tokens,name='embedding_layer'))

model.add(LSTM(units=16, return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(units=8, return_sequences=True))
model.add(Dropout(0.1))

model.add(LSTM(units=4))
model.add(Dropout(0.1))

model.add(Dense(1, activation='relu'))

optimizer = RMSProp()

model.compile(loss='mean_squared_error',optimizer=optimizer,metrics=['accuracy'])

In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_layer (Embedding)  (None, 68, 50)            750000    
_________________________________________________________________
lstm (LSTM)                  (None, 68, 16)            4288      
_________________________________________________________________
module_wrapper (ModuleWrappe (None, 68, 16)            0         
_________________________________________________________________
lstm_1 (LSTM)                (None, 68, 8)             800       
_________________________________________________________________
module_wrapper_1 (ModuleWrap (None, 68, 8)             0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 4)                 208       
_________________________________________________________________
module_wrapper_2 (ModuleWrap (None, 4)                 0

In [ ]:
history = model.fit(x_train_pad, y_train, validation_split=0.3, epochs=40, batch_size=100, shuffle=True, verbose = 1)

Epoch 1/40
206/206 [==============================] - 54s 214ms/step - loss: 0.4938 - accuracy: 0.5347 - val_loss: 0.3299 - val_accuracy: 0.5711
Epoch 2/40
206/206 [==============================] - 41s 197ms/step - loss: 0.3437 - accuracy: 0.5727 - val_loss: 0.3235 - val_accuracy: 0.5689
Epoch 3/40
206/206 [==============================] - 44s 214ms/step - loss: 0.3142 - accuracy: 0.5790 - val_loss: 0.3149 - val_accuracy: 0.5703
Epoch 4/40
206/206 [==============================] - 42s 205ms/step - loss: 0.3008 - accuracy: 0.5848 - val_loss: 0.3160 - val_accuracy: 0.5691
Epoch 5/40
206/206 [==============================] - 41s 201ms/step - loss: 0.2835 - accuracy: 0.5899 - val_loss: 0.3172 - val_accuracy: 0.5679
Epoch 6/40
206/206 [==============================] - 40s 195ms/step - loss: 0.2718 - accuracy: 0.5974 - val_loss: 0.3305 - val_accuracy: 0.5621
Epoch 7/40
206/206 [==============================] - 40s 194ms/step - loss: 0.2557 - accuracy: 0.6020 - val_loss: 0.3280 - val_ac

In [ ]:
result = model.evaluate(x_test_pad, y_test)

230/230 [==============================] - 5s 23ms/step - loss: 0.4800 - accuracy: 0.5338


**BERT**

In [ ]:
train, test= train_test_split(data_copy, test_size=0.2, random_state=42)
Xtrain, ytrain = train['discourse_text'], train['label']
Xtest, ytest = test['discourse_text'], test['label']
#splitting the train set into train and validation
Xtrain,Xval,ytrain,yval=train_test_split(Xtrain,ytrain,test_size=0.2,random_state=10)

In [ ]:
#set up the tokenizer
MAX_VOCAB_SIZE = 10000
tk = Tokenizer(num_words = MAX_VOCAB_SIZE,oov_token="<oov>")
tk.fit_on_texts(Xtrain)
word_index = tk.word_index
#print(word_index)
V = len(word_index)
print("Vocabulary of the dataset is : ",V)

Vocabulary of the dataset is :  21523


In [ ]:
##create sequences of reviews
seq_train = tk.texts_to_sequences(Xtrain)
seq_test =  tk.texts_to_sequences(Xtest)

In [ ]:
#choice of maximum length of sequences
seq_len_list = [len(i) for i in seq_train + seq_test]

#if we take the direct maximum then
max_len=max(seq_len_list)
print('Maximum length of sequence in the list: {}'.format(max_len))

Maximum length of sequence in the list: 379


In [ ]:
# when setting the maximum length of sequence, variability around the average is used.
max_seq_len = np.mean(seq_len_list) + 2 * np.std(seq_len_list)
max_seq_len = int(max_seq_len)
print('Maximum length of the sequence when considering data only two standard deviations from average: {}'.format(max_seq_len))

Maximum length of the sequence when considering data only two standard deviations from average: 69


In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.5 MB 5.3 MB/s 
     |████████████████████████████████| 163 kB 46.1 MB/s 
     |████████████████████████████████| 7.6 MB 39.5 MB/s 


In [ ]:
import transformers
#Perform tokenization
# automatically download the vocab used during pretraining or fine-tuning a given model,use from_pretrained() method
tokenizer = transformers.AutoTokenizer.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [ ]:
#pass our texts to the tokenizer. 
Xtrain_enc = tokenizer(Xtrain.tolist(), max_length=max_seq_len, 
                         truncation=True, padding='max_length', 
                         add_special_tokens=True, return_tensors='np') #return numpy object
Xval_enc = tokenizer(Xval.tolist(), max_length=max_seq_len, 
                         truncation=True, padding='max_length', 
                         add_special_tokens=True, return_tensors='np') #return numpy object
Xtest_enc = tokenizer(Xtest.tolist(), max_length=max_seq_len, 
                         truncation=True, padding='max_length', 
                         add_special_tokens=True, return_tensors='np') #return numpy object

In [ ]:
import tensorflow as tf
#preparing our datasets
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(Xtrain_enc),
    ytrain
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(Xval_enc),
    yval
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(Xtest_enc),
    ytest
))

In [ ]:
# creating BERT Model
from tensorflow.keras.layers import Dense,Input, Embedding,LSTM,Dropout,Conv1D
from tensorflow.keras.models import Model
def bert_model(train_dataset,val_dataset,transformer,max_len,epochs):
    print("----Building the model----")
    input_ids = Input(shape=(max_len,), dtype=tf.int32, name="input_ids")
    attention_mask = Input(shape=(max_len,),dtype=tf.int32,name = 'attention_mask') #attention mask
    sequence_output = transformer(input_ids,attention_mask)[0]
    cls_token = sequence_output[:, 0, :]
    x = Dense(512, activation='softmax')(cls_token)
    x = Dropout(0.1)(x)
    y = Dense(3, activation='relu')(x)
    model = Model(inputs=[input_ids,attention_mask], outputs=y)
    model.summary()
    model.compile(Adam(lr=2e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    r = model.fit(train_dataset.batch(120),batch_size = 120,
                  validation_data = val_dataset.batch(120),epochs = epochs)
                  #callbacks = callbacks
    print("Train score:", model.evaluate(train_dataset.batch(120)))
    print("Validation score:", model.evaluate(val_dataset.batch(120)))
    n_epochs = len(r.history['loss'])
    
    return r,model,n_epochs 

In [ ]:
transformer = transformers.TFDistilBertModel.from_pretrained('distilbert-base-uncased')

Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertModel: ['vocab_projector', 'vocab_transform', 'vocab_layer_norm', 'activation_13']
- This IS expected if you are initializing TFDistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFDistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFDistilBertModel for predictions without further training.


In [ ]:
epochs = 2
max_len = max_seq_len
r,model,n_epochs = bert_model(train_dataset,val_dataset,transformer,max_len,epochs)

----Building the model----
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_ids (InputLayer)         [(None, 69)]         0           []                               
                                                                                                  
 attention_mask (InputLayer)    [(None, 69)]         0           []                               
                                                                                                  
 tf_distil_bert_model (TFDistil  TFBaseModelOutput(l  66362880   ['input_ids[0][0]',              
 BertModel)                     ast_hidden_state=(N               'attention_mask[0][0]']         
                                one, 69, 768),                                                    
                                 hidden_states=None                